# UNL Crime Music
This program allows the conversion of University of Nebraska-Lincoln Police Department data into MIDI sounds. UNLPD's [Daily Crime and Fire Log](https://scsapps.unl.edu/policereports/MainPage.aspx) contains data back to 2006, which can be downloaded as a CSV using the "Advanced Search" option.  

This program relies heavily on the [Center for Investigative Reporting's](https://www.revealnews.org/blog/turn-your-data-into-sound-using-our-new-miditime-library/) [MIDITime library](https://github.com/cirlabs/miditime) to convert the data into a MIDI file. It also uses Chris Groskopf's [agate](https://github.com/wireservice/agate) library to manipulate the data by filtering and sorting it. 

In [1]:
import agate
from miditime.miditime import MIDITime
from collections import defaultdict

choose_month( ) filters the dataset by year and month.

In [2]:
def choose_month(table, column, year, month):
    # .where() method filters data
    # row[column] should be a datetime object, so .year and .month retrieve the year and month values, respectively
    return table.where(lambda row: (row[column].year==year) and (row[column].month==month))

sort_codes( ) groups the crimes by their code, counts them up and puts the count in a new column.

In [3]:
def sort_codes(table, column):
    # .group_by() returns a TableSet object
    by_code = table.group_by(column)
    # .aggregate() runs through a TableSet object and performs an aggregation
    codes = by_code.aggregate([('count', agate.Count())])
    # returns a table with just the codes and their counts, sorted from highest to lowest
    return codes.order_by('count', reverse=True)

convert_date( ) converts the date field into an decimal, in this case using MIDITime's days_since_epoch( ) method. The decimal is saved to a new column.

In [4]:
def convert_date(table, column):
    new_table = table.compute([
        ('Converted Date', agate.Formula(
            # days_since_epoch() takes a datetime object and calculates the number of days since Jan. 1, 1970
            # The MIDITime library needs the date to be a decimal value to appropriately scale events on the timeline
            agate.Number(), lambda row: MIDITime().days_since_epoch(row[column])
        )),
    ])
    return new_table

count_to_pitch( ) finds the appropriate pitch by finding the percentage of a number from a maximum and scaling it according to the midi_instance's range. A C pentatonic scale was used to drive down dissonance.

In [5]:
def count_to_pitch(row, column, max_count, midi_instance):
    # .linear_scale_pct() plots values between a minimum and maximum linearly 
    scale_pct = midi_instance.linear_scale_pct(0, float(max_count), float(row[column]), True)
    c_pent = ['C','D','E','G','A']
    # .scale_to_note() and .note_to_midi_pitch() then use that percent and the midi_instance's base octave and range to find the appropriate note
    note = midi_instance.scale_to_note(scale_pct, c_pent)
    midi_pitch = midi_instance.note_to_midi_pitch(note)
    return midi_pitch

code_to_pitch( ) takes a table of codes with their counts and calls count_to_pitch( ) on each row, creating a new column with the MIDI pitch value.

In [6]:
def code_to_pitch(table, column, midi_instance):
    # The max_count is calculated here so it is only calculated once
    max_count = table.aggregate(agate.Max(column))
    new_table = table.compute([
        ('Midi Pitch', agate.Formula(
            # Uses count_to_pitch to calculate the pitch for each row
            agate.Number(), lambda row: count_to_pitch(row, column, max_count, midi_instance) 
        )),
    ])
    return new_table

collect_notes( ) goes through each row of the crime table and assigns a MIDI pitch. If a lookup table is provided, it's used. Otherwise, the pitch 60 is used for all notes.

In [7]:
def collect_notes(crime_table, time_column, crime_column, midi_instance, pitch_table=None, pitch_column=None):
    # A defaultdict is used to prevent key errors when adding crime events
    note_list = defaultdict(lambda: [])
    # The start_time is set to the first crime in the dataset
    start_time = midi_instance.beat(float(crime_table.rows[0][time_column]))
    for row in crime_table.rows:
        crime_code = row[crime_column]
        # .beat() calculates the beat using the converted time and the midi_instance's secs/year value
        crime_time = midi_instance.beat(float(row[time_column]))
        if pitch_column and pitch_table:
            pitch = int(pitch_table.columns[pitch_column][crime_code])
        else:
            pitch = 60
        # The crime event is added to the note list under its crime code
        note_list[crime_code].append([crime_time-start_time, pitch, 100, 1])
    return note_list

allcrime_onemonth( ) collects all of the crimes that occured over one month, assigning each crime to a different track and channel.

In [8]:
def allcrime_onemonth(dataset, time_column, crime_column, year, month, midi_instance):
    # filter the data to one month
    month_slice = choose_month(dataset, time_column, year, month)
    # convert the date column to a decimal
    month_converted_date = convert_date(month_slice, time_column)
    # collect the codes and count them up
    crime_codes = sort_codes(month_slice, crime_column)
    # assign the pitches to the codes
    code_pitches = code_to_pitch(crime_codes, 'count', midi_instance)
    # assign the pitches to the events
    note_list = collect_notes(month_converted_date, 'Converted Date', crime_column, midi_instance, pitch_column='Midi Pitch', pitch_table=code_pitches)
    # create a new track for each crime type and add the notes
    for track_name in note_list:
        midi_instance.add_track(note_list[track_name], track_name)
    # save the file
    midi_instance.save_midi()
    return

find_common_codes( ) takes two months of data and finds the common codes for comparison, or just takes one set of data and returns the list of codes.

In [9]:
def find_common_codes(month_slice1, crime_column, month_slice2=None):
    # use sort_codes() to group by codes
    crime_codes1 = sort_codes(month_slice1, crime_column)
    codes_list1 = []
    for row in crime_codes1.rows:
        # There was a weird null value in one of the months, so now there's this
        if row[0] != None:
            codes_list1.append(row[0])
    if month_slice2:
        crime_codes2 = sort_codes(month_slice2, crime_column)
        codes_list2 = []
        for row in crime_codes2.rows:
            if row[0] != None:
                codes_list2.append(row[0])
        # This finds the common values between the two sets
        common_codes = list(set(codes_list1).intersection(codes_list2))
    else:
        common_codes = codes_list1
    return common_codes

pick_code( ) takes a list of codes, sorts them, prints them out and allows the user to pick one.

In [10]:
def pick_code(common_codes):
    sorted_codes = sorted(common_codes)
    for i, code in enumerate(sorted_codes):
        print("{0}: {1}".format(i, code))
    picked_number = int(input("Pick a number. "))
    picked_code = sorted_codes[picked_number]
    return picked_code

filter_crimes( ) takes a dataset and filters only the crimes based on a crime code.

In [11]:
def filter_crimes(dataset, crime_column, crime_code):
    filtered = dataset.where(lambda row: row[crime_column] == crime_code)
    return filtered

samecrime_twomonths( ) takes two months and presents the common crime codes. Once a code is chosen, the crimes are filtered and a MIDI file is created with the two months on different tracks.

In [12]:
def samecrime_twomonths(dataset, time_column, crime_column, year1, month1, year2, month2, midi_instance):
    # get two months of data
    month_slice1 = choose_month(dataset, time_column, year1, month1)
    month_slice2 = choose_month(dataset, time_column, year2, month2)
    # find their common crime codes
    common_codes = find_common_codes(month_slice1, crime_column, month_slice2=month_slice2)
    # have the user pick a code
    picked_code = pick_code(common_codes)
    # filter the data for that crime
    filtered1 = filter_crimes(month_slice1, crime_column, picked_code)
    filtered2 = filter_crimes(month_slice2, crime_column, picked_code)
    # convert the date column
    converted1 = convert_date(filtered1, time_column)
    converted2 = convert_date(filtered2, time_column)
    # collect the notes
    note_list1 = collect_notes(converted1, 'Converted Date', crime_column, midi_instance)
    note_list2 = collect_notes(converted2, 'Converted Date', crime_column, midi_instance)
    # add the notes to the midi instance, each month being a different track
    for track_name in note_list1:
        midi_instance.add_track(note_list1[track_name], "{0}/{1}".format(month1, year1))
    for track_name in note_list2:
        midi_instance.add_track(note_list2[track_name], "{0}/{1}".format(month2, year2))
    # save the file
    midi_instance.save_midi()
    return

onecrime_allmonths( ) filters the entire dataset for one crime and turns all of the events of that crime into MIDI.

In [13]:
def onecrime_allmonths(dataset, time_column, crime_column, midi_instance):
    # collect the codes from the dataset
    codes = find_common_codes(dataset, crime_column)
    # have the user pick a code
    picked_code = pick_code(codes)
    # filter the data for that crime
    filtered = filter_crimes(dataset, crime_column, picked_code)
    # convert the date column
    converted = convert_date(filtered, time_column)
    # collect the notes
    note_list = collect_notes(converted, 'Converted Date', crime_column, midi_instance)
    # add the notes to the midi instance
    for track_name in note_list:
        midi_instance.add_track(note_list[track_name], track_name)
    # save the file
    midi_instance.save_midi()
    return

Load the dataset.

In [14]:
all_crime = agate.Table.from_csv('all_years.csv')

Create an instance of the MIDITime class for each function. The third instance has a more condensed scale as its dataset is over more time.

In [15]:
# tempo, output destination, secs/year, base octave, range
midi_instance = MIDITime(120.0, 'allcrime_onemonth.mid', 182.5, 3, 5)

In [16]:
midi_instance2 = MIDITime(120.0, 'samecrime_twomonths.mid', 182.5, 3, 5)

In [17]:
midi_instance3 = MIDITime(120.0, 'onecrime_allmonths.mid', 18.25, 3, 5)

Run allcrime_onemonth( ).

In [18]:
allcrime_onemonth(all_crime, 'Reported', 'Incident Code', 2015, 5, midi_instance)

91 23.590000000000146 1 100
91 30.790000000000873 1 100
86 0.0 1 100
86 15.06000000000131 1 100
86 17.81000000000131 1 100
86 18.970000000001164 1 100
86 23.06000000000131 1 100
86 29.0 1 100
86 30.850000000002183 1 100
86 6.480000000003201 1 100
86 7.0900000000001455 1 100
86 8.530000000002474 1 100
86 8.580000000001746 1 100
86 13.06000000000131 1 100
86 16.210000000002765 1 100
86 23.610000000000582 1 100
88 3.7600000000020373 1 100
88 8.880000000001019 1 100
88 17.510000000002037 1 100
88 18.55000000000291 1 100
88 18.55000000000291 1 100
93 24.06000000000131 1 100
86 1.7400000000016007 1 100
86 9.030000000002474 1 100
86 13.520000000000437 1 100
86 14.340000000000146 1 100
86 14.760000000002037 1 100
86 15.700000000000728 1 100
86 1.2200000000011642 1 100
86 4.850000000002183 1 100
86 7.540000000000873 1 100
86 11.070000000003347 1 100
86 14.720000000001164 1 100
86 27.780000000002474 1 100
91 5.010000000002037 1 100
91 7.080000000001746 1 100
91 30.600000000002183 1 100
91 1.1600

Run samecrime_twomonths( ).

In [19]:
samecrime_twomonths(all_crime, 'Reported', 'Incident Code', 2014, 10, 2015, 10, midi_instance2)

0: ACCIDENTS - P.D. H&R NOT REPORTABLE
1: ACCIDENTS - P.D. NOT REPORTABLE
2: ACCIDENTS - P.D. REPORTABLE
3: ALCOHOL - DRUNK
4: ALCOHOL - MINOR IN POSSESSION
5: DISTURBANCE - OTHER
6: FALSE INFORMATION - GIVEN TO OFC.
7: FIRE - FALSE ALARM
8: FRAUD - CREDIT CARDS/ATM/BANK CARD
9: LARCENY - FROM BUILDING
10: LARCENY - OTHER OR FROM OPEN AREA
11: LARCENY - STOLEN BIKE
12: LOST OR STOLEN ITEM
13: MEDICAL EMERGENCY
14: MISC - OTHER
15: NARCOTICS - OTHER
16: NARCOTICS - POSSESSION
17: PROPERTY DAMAGE - UNINTENTIONAL,NON TRAFFIC
18: SS - CHECK WELFARE OF PERSON
19: SUSPICIOUS PERSON
20: TRAFFIC - OTHER
21: TRAFFIC - SUSPENDED DRIVER
22: TRESPASSING
23: VANDALISM - OTHER
Pick a number. 22
60 0.0 1 100
60 1.25 1 100
60 9.409999999999854 1 100
60 20.5 1 100
60 26.529999999998836 1 100
60 0.0 1 100
60 2.0499999999992724 1 100
60 8.069999999999709 1 100
60 19.31999999999971 1 100
60 21.31999999999971 1 100


Run onecrime_allmonths( ).

In [20]:
onecrime_allmonths(all_crime, 'Reported', 'Incident Code', midi_instance3)

0: ABANDONED BICYCLE
1: ACCIDENTS - FATALITY
2: ACCIDENTS - INJURY
3: ACCIDENTS - INJURY H&R
4: ACCIDENTS - P.D. H&R NOT REPORTABLE
5: ACCIDENTS - P.D. H&R REPORTABLE
6: ACCIDENTS - P.D. NOT REPORTABLE
7: ACCIDENTS - P.D. REPORTABLE
8: ALCOHOL - BUY/PROCURE FOR MINORS
9: ALCOHOL - CONSUMPTION IN PUBLIC
10: ALCOHOL - DRUNK
11: ALCOHOL - DWI
12: ALCOHOL - LIQUOR LICENSE VIOLATION
13: ALCOHOL - MINOR IN POSSESSION
14: ALCOHOL - OPEN CONTAINER
15: ALCOHOL - POSSESSION IN PARK
16: ALCOHOL - UNL POLICY VIOLATION
17: ALCOHOL - USING FALSE I.D.
18: ANIMAL - All CALLS OTHER THAN DOGS
19: ANIMAL - DOG BITE TO PERSON
20: ANIMAL - DOG RUNNING LOOSE
21: ANIMAL - OTHER DOG CALL
22: ARSON
23: ASSAULT - DOMESTIC
24: ASSAULT - NON DOMESTIC
25: ASSAULT - OF POLICE OFFICER
26: AUTO THEFT
27: AUTO THEFT - STOLEN ELSEWHERE, FOUND IN  LINCOLN
28: BOMB - EXPLOSION
29: BOMB - SUSPICIOUS PACKAGE/DEVICE
30: BOMB - THREAT
31: BURGLARY
32: CHILD ABUSE/NEG - PHYSICAL ABUSE
33: CHILD ABUSE/NEG - UNATTENDED IN VEH
3